## Evaluation
* We just evaluate everything without worrying about alignment
* Make sure to run `Preparation.ipynb` before.

In [1]:
import shutil
import os
os.makedirs('tmp_results', exist_ok=True)

In [2]:
from scripts.data_management import EuroParlManager, FloresPlusManager
dms = {
    'ep' : EuroParlManager(),
    'flores' : FloresPlusManager()
}

tls = ['gpt', 'deepl']

In [3]:
all_pairs = EuroParlManager.get_pairs()


In [ ]:
from scripts.post_process import direct_triplet_align, load_mt_sents
from os.path import join
for pair in all_pairs:
    s, t = pair
    for dataset in dms:
        if dataset == 'ep' and pair == ('it', 'el'):
            continue
        for translator in tls:
            mt_sents = load_mt_sents(dataset, translator, s, t)
            src_sents, tgt_sents = dms[dataset].get_sentence_pairs(s, t, num_of_sents=400)
            direct_triplet_align(
                mt_sents=mt_sents,
                ref_sents=tgt_sents,
                src_sents=src_sents,
                src_lang=s,
                ref_lang=t,
                folder_path=join('tmp_results', f'{dataset}-{translator}')
            )

In [7]:
from scripts.scoring import ResultProducer
aligned = os.listdir('tmp_results')
for res in aligned:
    fp = join('tmp_results', res)
    files = os.listdir(fp)
    l2f = {f.replace('.jsonl', ''): join(fp, f) for f in files}
    rp = ResultProducer(label2files=l2f)
    rp.compute_results()
    rp.store_results(join('tmp_results', f'{res}.csv'))



In [10]:
from scripts.scoring import create_matrix_from_csv
ep_gpt = create_matrix_from_csv(join('tmp_results', 'ep-gpt.csv'))
ep_deepl = create_matrix_from_csv(join('tmp_results', 'ep-deepl.csv'))
flores_gpt = create_matrix_from_csv(join('tmp_results', 'flores-gpt.csv'))
flores_deepl = create_matrix_from_csv(join('tmp_results', 'flores-deepl.csv'))

### Matrices
* From source to target (left to right), the row is the source and the column is the target

In [16]:
ep_gpt

,da,de,el,en,es,fi,fr,it,nl,pt,sv
da,NaN,34.052207,27.403131,34.265758,33.450159,21.559752,31.853186,19.909345,26.493696,26.156095,29.329501
de,34.496495,NaN,23.401336,7.483574,34.405740,4.767343,31.288819,24.302457,25.314173,27.390053,27.251319
el,32.528914,27.909468,NaN,34.108317,37.581394,19.816198,36.058423,11.543177,12.862619,29.987619,27.174453
en,34.598924,27.088722,28.540416,NaN,36.721217,5.877664,33.207951,27.096256,27.868022,23.898661,27.720709
es,36.416038,32.385556,15.403091,2.409827,NaN,19.211941,35.952666,28.261688,15.050712,32.150601,28.047307
fi,28.984256,26.403350,22.939204,31.805305,29.777864,NaN,30.513101,7.239586,6.729594,24.917295,24.411380
fr,33.007701,28.890542,28.057027,33.892693,37.496366,18.709931,NaN,27.078451,27.220359,27.383797,27.204276
it,26.915048,22.830478,NaN,29.144336,31.736969,13.175652,28.628564,NaN,23.233386,25.929821,21.734194
nl,28.484463,22.562986,21.464066,29.693938,27.445810,16.068469,26.936332,22.018028,NaN,22.855778,22.578141
pt,32.535884,28.399012,28.751309,33.078499,35.746317,5.846544,33.797273,25.771903,25.961316,NaN,24.433565


In [12]:
ep_deepl

,da,de,el,en,es,fi,fr,it,nl,pt,sv
da,NaN,34.060691,28.320274,40.892690,37.117352,25.503815,35.695082,27.567300,30.864241,31.047434,31.234542
de,34.974381,NaN,26.427654,37.683698,36.148667,24.586250,37.804653,27.186651,28.339887,30.970712,29.996975
el,34.401319,29.992630,NaN,39.020138,38.186257,23.645183,37.261932,28.691005,28.528897,34.262699,29.280422
en,37.497330,32.547469,30.799290,NaN,41.137497,24.369398,38.496851,29.162393,32.202903,34.195964,32.251568
es,37.065578,32.860527,30.206930,43.274294,NaN,24.895837,39.912205,30.341695,30.271553,35.809157,30.512634
fi,32.623281,28.696167,25.109554,35.488300,32.030200,NaN,32.506162,24.320832,26.955631,27.897197,26.504494
fr,32.871805,30.348471,27.817182,38.308544,37.703949,22.997572,NaN,28.788980,29.003367,33.227476,28.527832
it,29.072912,26.528497,NaN,33.204503,34.749799,19.833691,33.320046,NaN,26.338769,29.982963,24.206840
nl,28.717135,25.253817,22.129751,31.239803,29.104525,19.464109,29.808660,23.729883,NaN,26.387243,23.574642
pt,32.717717,30.738889,28.661167,37.206175,39.076059,23.158269,39.056324,28.814821,28.687616,NaN,27.938150


In [13]:
flores_gpt

,da,de,el,en,es,fi,fr,it,nl,pt,sv
da,NaN,38.863723,25.144143,51.678229,26.768960,25.403951,43.613682,29.552197,28.913497,42.015165,38.968640
de,38.691221,NaN,24.628724,48.811945,26.089909,25.572269,40.467246,29.130906,28.881781,38.349227,36.044331
el,34.773403,32.386091,NaN,43.565306,25.183134,22.467562,39.093314,25.621634,26.434385,35.889013,32.381023
en,49.252663,43.614308,28.990484,NaN,29.179662,29.367920,51.947871,32.661039,30.930165,51.370512,46.351554
es,28.640257,28.275379,0.228810,35.985613,NaN,19.249954,32.255087,21.654174,23.112735,24.278693,27.263133
fi,0.293491,29.987002,0.217186,39.660204,0.650015,NaN,36.203299,25.802701,0.599517,0.583201,27.622318
fr,36.655553,34.863355,24.748256,49.696842,26.638170,24.417397,NaN,26.510702,26.660309,39.963644,35.290419
it,29.439101,31.180642,21.067091,39.206555,22.630972,20.852346,0.622156,NaN,23.426438,29.244247,28.651399
nl,28.519843,28.786491,18.973567,36.991055,23.369889,18.381937,33.125362,24.152701,NaN,30.540325,25.760948
pt,39.132374,36.787979,24.158032,55.545055,25.336856,24.432898,44.780088,27.240783,27.576622,NaN,36.774190


In [14]:
flores_deepl

,da,de,el,en,es,fi,fr,it,nl,pt,sv
da,NaN,37.903019,27.065014,54.799530,26.358653,26.373987,44.505666,30.947992,29.349099,34.960044,39.809583
de,41.110841,NaN,25.357988,49.230138,24.867585,25.864145,41.499288,30.679027,28.574291,33.325661,37.334801
el,34.583404,31.615394,NaN,41.849369,24.529222,21.634200,38.631227,27.965881,24.619949,29.797411,31.002461
en,50.610947,44.393833,30.589381,NaN,28.737538,29.775069,52.424001,34.797073,32.372457,42.252638,47.076802
es,30.376231,27.618031,20.542459,35.793684,NaN,19.905015,35.569537,27.183703,24.408720,27.168136,27.571089
fi,33.174106,30.691418,21.968231,38.213989,21.449547,NaN,36.977580,26.401138,24.790704,27.879875,30.116650
fr,38.416856,34.551130,25.433119,49.227220,26.064141,24.930215,NaN,30.600824,27.320816,33.634741,35.322091
it,31.286225,29.542371,22.127380,37.348334,24.232551,20.625066,36.913005,NaN,23.889913,27.824569,29.052660
nl,31.393276,29.895214,20.475395,36.490951,23.124973,21.546739,35.004903,26.151200,NaN,27.540381,28.197053
pt,39.723654,36.616311,26.257181,53.701485,25.878094,24.627075,44.189374,31.001435,27.102937,NaN,37.075581


## Evaluation After Alignment
* Concerns GPT4.1 only

In [4]:
from scripts.scoring import ResultProducer
from os.path import join
import os
dm2l2f = {}
aligned = [f for f in os.listdir('tmp_results') if not f.endswith('.csv')]
for res in aligned:
    if 'gpt' in res:
        fp = join('tmp_results', res)
        files = os.listdir(fp)
        l2f = {f.replace('.jsonl', ''): join(fp, f) for f in files}
        dm2l2f[res] = l2f


In [5]:
re_aligned = {
    'flores': join('triplets', 'flores'),
    'ep': join('triplets', 'ep')
}

for key in dm2l2f:
    dm, tl = key.split('-')
    files = os.listdir(re_aligned[dm])
    print(len(files))
    for f in files:
        l2fkey = f.replace('.jsonl', '')
        dm2l2f[key][l2fkey] = join(re_aligned[dm], f)

109
110


In [6]:
for key in dm2l2f:
    rp = ResultProducer(label2files=dm2l2f[key])
    rp.compute_results()
    rp.store_results(join('tmp_results', f'{key}-new.csv'))

In [7]:
from scripts.scoring import create_matrix_from_csv
ep_gpt = create_matrix_from_csv(join('tmp_results', 'ep-gpt-new.csv'))
flores_gpt = create_matrix_from_csv(join('tmp_results', 'flores-gpt-new.csv'))

In [8]:
ep_gpt

,da,de,el,en,es,fi,fr,it,nl,pt,sv
da,NaN,34.370946,27.226428,34.769287,33.360553,22.103023,32.065179,25.182416,27.095604,26.384279,29.764738
de,34.923509,NaN,23.498233,32.922238,35.042658,20.678602,31.271454,24.960307,25.802126,27.208893,28.182273
el,32.673476,28.348979,NaN,34.431766,37.856485,20.006031,36.533975,26.296305,25.520287,30.196619,27.444643
en,34.791041,27.711227,28.894360,NaN,37.077636,19.360128,33.598053,26.785969,28.407580,24.308233,28.484771
es,36.533510,32.717835,30.587555,36.749479,NaN,19.564808,35.973488,28.559097,27.624149,32.345052,28.335849
fi,29.136995,26.714491,22.760685,31.593031,29.932740,NaN,30.381735,22.433048,22.691877,24.684268,24.671074
fr,33.086890,28.849903,27.856996,34.095530,37.663445,18.445387,NaN,26.906858,27.174163,27.389646,27.324933
it,27.187949,23.597261,NaN,29.513785,32.465649,15.750369,29.235352,NaN,23.679009,26.652081,22.332657
nl,30.984333,24.054102,23.026105,32.032896,30.069953,17.905032,29.111434,23.610502,NaN,24.346593,25.050300
pt,32.879224,28.582109,28.723902,33.341236,35.602202,19.012961,34.095221,26.337313,26.594406,NaN,24.713030


In [9]:
flores_gpt

,da,de,el,en,es,fi,fr,it,nl,pt,sv
da,NaN,38.723192,25.380582,51.612687,26.931556,25.479791,43.836579,29.616165,28.882634,42.124911,38.970147
de,38.936773,NaN,24.967368,48.963555,26.409032,26.008855,41.030546,29.324952,29.177627,38.222157,36.382156
el,34.650895,32.342131,NaN,43.519187,25.424947,22.464683,39.091221,25.638073,26.520976,35.751700,32.487665
en,49.187001,43.616338,28.944465,NaN,29.203995,29.355850,51.769676,32.590283,30.851282,51.193180,46.292829
es,28.791149,28.489824,19.872813,36.168866,NaN,19.340911,32.433983,21.702213,23.289135,24.359257,27.349275
fi,30.449397,30.080374,21.361150,40.222993,23.086827,NaN,36.602554,25.805762,25.389318,32.681139,27.779270
fr,36.805764,35.072333,24.959154,49.781756,26.747322,24.455008,NaN,26.487884,26.670731,40.099356,35.354419
it,29.421292,30.987880,21.065624,39.551184,22.665812,20.858643,33.369091,NaN,23.531327,29.237774,28.481378
nl,28.302400,28.497524,19.155999,37.252077,23.534409,18.833439,33.370438,23.940471,NaN,30.863328,25.530058
pt,39.339296,37.002434,24.283239,55.845420,25.456757,24.562808,45.060996,27.357773,27.757053,NaN,36.912072
